## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Katsuura,JP,35.1333,140.3000,75.24,92,100,14.27,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,74.71,78,59,13.94,light rain
2,2,Ushuaia,AR,-54.8000,-68.3000,29.86,74,75,18.41,light snow
3,3,Tavda,RU,58.0425,65.2726,65.35,86,20,4.52,few clouds
4,4,Carnarvon,AU,-24.8667,113.6333,66.27,100,75,0.00,broken clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Katsuura,JP,35.1333,140.3000,75.24,92,100,14.27,light rain
6,6,Barcelos,PT,41.5388,-8.6151,81.68,56,0,1.95,clear sky
8,8,Devils Lake,US,48.1128,-98.8651,84.31,58,0,18.41,clear sky
9,9,Georgetown,MY,5.4112,100.3354,84.13,92,20,3.44,moderate rain
11,11,Katubao,PH,5.8892,124.8294,77.16,91,99,10.18,overcast clouds
12,12,Milkovo,RU,54.7167,158.6167,75.81,74,100,1.57,overcast clouds
19,19,Lorengau,PG,-2.0226,147.2712,81.61,80,95,11.61,overcast clouds
22,22,Albany,US,42.6001,-73.9662,81.52,40,2,4.70,clear sky
25,25,Cabatuan,PH,10.8794,122.4860,76.10,91,100,9.60,overcast clouds
27,27,Hithadhoo,MV,-0.6000,73.0833,82.24,76,36,6.22,scattered clouds


In [24]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                 0
City                    0
Country                 0
Lat                     0
Lng                     0
Max Temp                0
Humidity                0
Cloudiness              0
Wind Speed              0
Current Description     0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description ", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Katsuura,JP,75.24,light rain,35.1333,140.3000,
6,Barcelos,PT,81.68,clear sky,41.5388,-8.6151,
8,Devils Lake,US,84.31,clear sky,48.1128,-98.8651,
9,Georgetown,MY,84.13,moderate rain,5.4112,100.3354,
11,Katubao,PH,77.16,overcast clouds,5.8892,124.8294,
12,Milkovo,RU,75.81,overcast clouds,54.7167,158.6167,
19,Lorengau,PG,81.61,overcast clouds,-2.0226,147.2712,
22,Albany,US,81.52,clear sky,42.6001,-73.9662,
25,Cabatuan,PH,76.10,overcast clouds,10.8794,122.4860,
27,Hithadhoo,MV,82.24,scattered clouds,-0.6000,73.0833,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [21]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Katsuura,JP,75.24,light rain,35.1333,140.3000,Katsuura Hotel Mikazuki
6,Barcelos,PT,81.68,clear sky,41.5388,-8.6151,Hotel Restaurante Bagoeira
8,Devils Lake,US,84.31,clear sky,48.1128,-98.8651,"Holiday Inn Express Devils Lake, an IHG Hotel"
9,Georgetown,MY,84.13,moderate rain,5.4112,100.3354,Cititel Penang
11,Katubao,PH,77.16,overcast clouds,5.8892,124.8294,IML Ecopark
12,Milkovo,RU,75.81,overcast clouds,54.7167,158.6167,"Hotel ""Geologist"""
19,Lorengau,PG,81.61,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
22,Albany,US,81.52,clear sky,42.6001,-73.9662,
25,Cabatuan,PH,76.10,overcast clouds,10.8794,122.4860,MVC Transient Room
27,Hithadhoo,MV,82.24,scattered clouds,-0.6000,73.0833,Scoop Guest House


In [36]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Katsuura,JP,75.24,light rain,35.1333,140.3000,Katsuura Hotel Mikazuki
6,Barcelos,PT,81.68,clear sky,41.5388,-8.6151,Hotel Restaurante Bagoeira
8,Devils Lake,US,84.31,clear sky,48.1128,-98.8651,"Holiday Inn Express Devils Lake, an IHG Hotel"
9,Georgetown,MY,84.13,moderate rain,5.4112,100.3354,Cititel Penang
11,Katubao,PH,77.16,overcast clouds,5.8892,124.8294,IML Ecopark
12,Milkovo,RU,75.81,overcast clouds,54.7167,158.6167,"Hotel ""Geologist"""
19,Lorengau,PG,81.61,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
25,Cabatuan,PH,76.10,overcast clouds,10.8794,122.4860,MVC Transient Room
27,Hithadhoo,MV,82.24,scattered clouds,-0.6000,73.0833,Scoop Guest House
33,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description }</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))